## =============================================================================
## NOTEBOOK 01 - PRÉPARATION DES DONNÉES
## =============================================================================

## 📊 Préparation des Données pour Drone Agri AI

## Ce notebook couvre:
## 1. Téléchargement des datasets
## 2. Exploration et visualisation
## 3. Nettoyage et prétraitement
## 4. Augmentation des données
## 5. Division train/val/test

In [ ]:
# Configuration
!pip install -q kaggle albumentations matplotlib seaborn

import os
import sys
import json
import shutil
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import albumentations as A
from tqdm.notebook import tqdm

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Chemins
PROJECT_DIR = Path('/content/drone-agri-ai')
DATA_DIR = Path('/content/data')
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'

for d in [PROJECT_DIR, DATA_DIR, RAW_DIR, PROCESSED_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Project: {PROJECT_DIR}")
print(f"Data: {DATA_DIR}")

# Configuration Kaggle
# Méthode 1: Upload manuel
from google.colab import files
print("Uploadez votre fichier kaggle.json:")
# uploaded = files.upload()

# Méthode 2: Depuis Google Drive
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/ 2>/dev/null || echo "Fichier non trouvé dans Drive"
!chmod 600 ~/.kaggle/kaggle.json

# Vérifier
!kaggle datasets list -s plant --sort-by votes | head -10

# Télécharger les datasets
print("📥 Téléchargement des datasets...")

# PlantVillage - Principal dataset
!kaggle datasets download -d abdallahalidev/plantvillage-dataset -p {RAW_DIR} --unzip

# New Plant Diseases Dataset
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset -p {RAW_DIR} --unzip

# Plant Seedlings (optionnel)
# !kaggle datasets download -d vbookshelf/v2-plant-seedlings-dataset -p {RAW_DIR} --unzip

print("✅ Téléchargement terminé!")

In [ ]:
# Explorer la structure
def explore_directory(path, max_depth=3, current_depth=0):
    """Explore récursivement un répertoire"""
    if current_depth >= max_depth:
        return
    
    path = Path(path)
    indent = "  " * current_depth
    
    for item in sorted(path.iterdir())[:20]:  # Limiter à 20 items
        if item.is_dir():
            count = len(list(item.glob('*')))
            print(f"{indent}📁 {item.name}/ ({count} items)")
            explore_directory(item, max_depth, current_depth + 1)
        else:
            size = item.stat().st_size / 1024  # KB
            print(f"{indent}📄 {item.name} ({size:.1f} KB)")

print("Structure du dossier data:")
explore_directory(DATA_DIR, max_depth=3)

# Analyser PlantVillage
# Trouver le dossier des images
plantvillage_paths = list(RAW_DIR.glob('**/color')) + list(RAW_DIR.glob('**/Color'))
if not plantvillage_paths:
    plantvillage_paths = list(RAW_DIR.glob('**/*'))
    
print(f"Chemins trouvés: {plantvillage_paths[:5]}")

# Utiliser le premier chemin valide
data_path = None
for p in plantvillage_paths:
    if p.is_dir() and len(list(p.iterdir())) > 10:
        data_path = p
        break

if data_path is None:
    # Chercher directement les dossiers de classes
    for p in RAW_DIR.rglob('*'):
        if p.is_dir():
            subdirs = list(p.iterdir())
            if len(subdirs) > 20 and all(d.is_dir() for d in subdirs[:5]):
                data_path = p
                break

print(f"📂 Dossier de données: {data_path}")

# Statistiques des classes
classes = sorted([d.name for d in data_path.iterdir() if d.is_dir()])
print(f"\n📊 Nombre de classes: {len(classes)}")

# Compter les images par classe
class_counts = {}
for class_name in tqdm(classes, desc="Comptage"):
    class_dir = data_path / class_name
    count = len(list(class_dir.glob('*.jpg'))) + len(list(class_dir.glob('*.JPG'))) + len(list(class_dir.glob('*.png')))
    class_counts[class_name] = count

# DataFrame pour analyse
df_classes = pd.DataFrame([
    {"class": k, "count": v} for k, v in class_counts.items()
])
df_classes = df_classes.sort_values('count', ascending=False)

print(f"\n📈 Total d'images: {df_classes['count'].sum()}")
print(f"📈 Moyenne par classe: {df_classes['count'].mean():.0f}")
print(f"📈 Min: {df_classes['count'].min()}")
print(f"📈 Max: {df_classes['count'].max()}")

# Visualiser la distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Distribution des classes
ax1 = axes[0]
df_classes.plot(x='class', y='count', kind='bar', ax=ax1, legend=False)
ax1.set_title('Distribution des images par classe', fontsize=14)
ax1.set_xlabel('Classe')
ax1.set_ylabel('Nombre d\'images')
ax1.tick_params(axis='x', rotation=90, labelsize=6)

# Histogramme
ax2 = axes[1]
ax2.hist(df_classes['count'], bins=30, edgecolor='black')
ax2.set_title('Histogramme des tailles de classes', fontsize=14)
ax2.set_xlabel('Nombre d\'images')
ax2.set_ylabel('Fréquence')
ax2.axvline(df_classes['count'].mean(), color='red', linestyle='--', label='Moyenne')
ax2.legend()

plt.tight_layout()
plt.savefig(PROJECT_DIR / 'class_distribution.png', dpi=150)
plt.show()

# Parser les noms de classes
def parse_class_name(name):
    """Extrait la plante et la condition du nom de classe"""
    parts = name.split('___')
    if len(parts) == 2:
        return {
            'plant': parts[0].replace('_', ' '),
            'condition': parts[1].replace('_', ' '),
            'is_healthy': 'healthy' in parts[1].lower()
        }
    return {
        'plant': name,
        'condition': 'unknown',
        'is_healthy': None
    }

# Analyser toutes les classes
class_info = {}
plants = set()
conditions = set()

for class_name in classes:
    info = parse_class_name(class_name)
    class_info[class_name] = info
    plants.add(info['plant'])
    conditions.add(info['condition'])

print(f"\n🌱 Plantes uniques: {len(plants)}")
for p in sorted(plants):
    print(f"  - {p}")

print(f"\n🏥 Conditions uniques: {len(conditions)}")

In [ ]:
# Visualiser des exemples
def show_samples(data_path, classes, samples_per_class=3):
    """Affiche des exemples de chaque classe"""
    n_classes = min(12, len(classes))
    fig, axes = plt.subplots(n_classes, samples_per_class, figsize=(samples_per_class * 3, n_classes * 2.5))
    
    for i, class_name in enumerate(classes[:n_classes]):
        class_dir = data_path / class_name
        images = list(class_dir.glob('*.jpg')) + list(class_dir.glob('*.JPG'))
        
        for j in range(samples_per_class):
            ax = axes[i, j] if n_classes > 1 else axes[j]
            
            if j < len(images):
                img = cv2.imread(str(images[j]))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                ax.imshow(img)
            
            ax.axis('off')
            if j == 0:
                ax.set_title(class_name[:30], fontsize=8)
    
    plt.tight_layout()
    plt.savefig(PROJECT_DIR / 'sample_images.png', dpi=150)
    plt.show()

show_samples(data_path, classes[:12])

# Vérifier la qualité des images
print("🔍 Vérification de la qualité des images...")

issues = []
image_sizes = []
image_formats = Counter()

for class_name in tqdm(classes[:10], desc="Vérification"):  # Limiter pour rapidité
    class_dir = data_path / class_name
    
    for img_path in list(class_dir.glob('*'))[:50]:  # 50 images par classe
        try:
            img = cv2.imread(str(img_path))
            if img is None:
                issues.append(f"Cannot read: {img_path}")
                continue
            
            h, w = img.shape[:2]
            image_sizes.append((w, h))
            image_formats[img_path.suffix.lower()] += 1
            
            # Vérifier les dimensions minimales
            if w < 50 or h < 50:
                issues.append(f"Too small ({w}x{h}): {img_path}")
            
        except Exception as e:
            issues.append(f"Error {e}: {img_path}")

print(f"\n📊 Formats trouvés: {dict(image_formats)}")
print(f"⚠️ Problèmes détectés: {len(issues)}")

if issues:
    print("\nExemples de problèmes:")
    for issue in issues[:5]:
        print(f"  - {issue}")

# Statistiques de taille
sizes = np.array(image_sizes)
print(f"\n📐 Tailles d'images:")
print(f"  Min: {sizes.min(axis=0)}")
print(f"  Max: {sizes.max(axis=0)}")
print(f"  Moyenne: {sizes.mean(axis=0).astype(int)}")

In [ ]:
# Préparer le dataset final
print("📦 Préparation du dataset final...")

# Structure cible
TRAIN_DIR = PROCESSED_DIR / 'train'
VAL_DIR = PROCESSED_DIR / 'val'
TEST_DIR = PROCESSED_DIR / 'test'

for d in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    if d.exists():
        shutil.rmtree(d)
    d.mkdir(parents=True)

# Ratios
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

# Copier les images
np.random.seed(42)

for class_name in tqdm(classes, desc="Copie des images"):
    src_dir = data_path / class_name
    images = list(src_dir.glob('*.jpg')) + list(src_dir.glob('*.JPG')) + list(src_dir.glob('*.png'))
    
    np.random.shuffle(images)
    
    n = len(images)
    n_train = int(n * TRAIN_RATIO)
    n_val = int(n * VAL_RATIO)
    
    splits = {
        'train': images[:n_train],
        'val': images[n_train:n_train + n_val],
        'test': images[n_train + n_val:]
    }
    
    for split_name, split_images in splits.items():
        dest_dir = PROCESSED_DIR / split_name / class_name
        dest_dir.mkdir(parents=True, exist_ok=True)
        
        for img_path in split_images:
            shutil.copy2(img_path, dest_dir / img_path.name)

# Vérifier le split
def count_split(split_dir):
    """Compte les images dans un split"""
    total = 0
    for class_dir in split_dir.iterdir():
        if class_dir.is_dir():
            total += len(list(class_dir.glob('*')))
    return total

print("\n📊 Résumé du split:")
print(f"  Train: {count_split(TRAIN_DIR)} images")
print(f"  Val: {count_split(VAL_DIR)} images")
print(f"  Test: {count_split(TEST_DIR)} images")

# Sauvegarder les métadonnées
metadata = {
    "created_at": pd.Timestamp.now().isoformat(),
    "source": "PlantVillage",
    "num_classes": len(classes),
    "classes": classes,
    "class_info": class_info,
    "splits": {
        "train": count_split(TRAIN_DIR),
        "val": count_split(VAL_DIR),
        "test": count_split(TEST_DIR)
    },
    "plants": list(plants),
    "input_size": [224, 224]
}

with open(PROCESSED_DIR / 'metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✅ Métadonnées sauvegardées!")

# Copier sur Google Drive
!cp -r {PROCESSED_DIR}/* /content/drive/MyDrive/drone-agri-ai/data/
print("✅ Données copiées sur Google Drive!")

# ## ✅ Données prêtes!
# 
# Le dataset est maintenant prêt pour l'entraînement:
# - Images organisées en train/val/test
# - Métadonnées sauvegardées
# - Copie sur Google Drive